In [1]:
import pandas as pd              #Used for data manipulation and analysis
import requests                  #Allows us to send HTTP requests to websites to get the HTML content.
from bs4 import BeautifulSoup    # extracting data from website
import matplotlib.pyplot as plt  # data visualization 
import sqlite3                   # Used for creating and managing an SQLite database.

In [37]:
# Function to scrape product data from Amazon

def scrape_amazon():
    url = 'https://www.amazon.com/s?k=laptop'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    product_list = []
    for product in products[:5]:  # Limiting to 5 products for simplicity
        name = product.h2.text.strip()
        price = product.find('span', 'a-price-whole')
        if price:
            price = price.text.replace(',', '').replace('$', '').strip()
            product_list.append({'name': name, 'price': float(price)})
    
    return pd.DataFrame(product_list)



In [39]:
# Function to scrape product data from eBay

def scrape_ebay():
    url = 'https://www.ebay.com/sch/i.html?_nkw=laptop'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    products = soup.find_all('li', {'class': 's-item'})

    product_list = []
    for product in products[:5]:  # Limiting to 5 products for simplicity
        name_tag = product.find('h3', class_='s-item__title')
        price_tag = product.find('span', class_='s-item__price')
        if name_tag and price_tag:
            name = name_tag.text.strip()
            price = price_tag.text.replace(',', '').replace('$', '').strip().split()[0]
            try:
                product_list.append({'name': name, 'price': float(price)})
            except ValueError:
                continue
    
    return pd.DataFrame(product_list)

In [41]:
# Scrape data from both websites

data_amazon = scrape_amazon()
data_ebay = scrape_ebay()

In [44]:
# Check if both dataframes have the 'name' column

if 'name' in data_amazon.columns and 'name' in data_ebay.columns:
    # Merge dataframes on product names
    merged_data = pd.merge(data_amazon, data_ebay, on='name', how='outer', suffixes=('_amazon', '_ebay'))

In [46]:
# Plot the comparison

plt.figure(figsize =(10,6))
plt.plot(merged_data['name'], merged_data['price_amazon'], label='Amazon', marker='o')
plt.plot(merged_data['name'], merged_data['price_ebay'], label='eBay', marker='x')
plt.xlabel('Product Name')
plt.ylabel('Price')
plt.title('Price Comparison Between Amazon and eBay')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.savefig('/mnt/data/price_comparison.png')
plt.show()

NameError: name 'merged_data' is not defined

<Figure size 432x432 with 0 Axes>

In [30]:
   # Save the merged data to an SQLite database
    conn = sqlite3.connect('/mnt/data/ecommerce_data.db')
    merged_data.to_sql('product_prices', conn, if_exists='replace', index=False)

IndentationError: unexpected indent (3268095706.py, line 2)